<a href="https://colab.research.google.com/github/chaos44/food_recognition/blob/master/food.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)#2. Get the file
d10 = drive.CreateFile({'id':'1jqXr5d-UAhOUaPIb-g7p_8xEW9XxHjUr'}) # replace the id with id of file you want to access
d20 = drive.CreateFile({'id':'11LROdN-lQh0b25xoxNFBjSbYhQrEG5Dy'})
d30 = drive.CreateFile({'id':'12uU6ocY9r6BfBk8fo4BQC8BVXjDDhCBb'})
d40 = drive.CreateFile({'id':'1Hs5OVtYD8xDvLd7y6drDHUu4ErXjBvr_'})
d50 = drive.CreateFile({'id':'19FaeD7fdFlqu0U8K2DIJevtp1tb4PbBe'})
d10.GetContentFile('d10.zip')
d20.GetContentFile('d20.zip')
d30.GetContentFile('d30.zip')
d40.GetContentFile('d40.zip')
d50.GetContentFile('d50.zip')

In [ ]:
!unzip d10.zip -d food
!unzip d20.zip -d food
!unzip d30.zip -d food
!unzip d40.zip -d food
!unzip d50.zip -d food

In [1]:
from PIL import Image
import os, glob
import numpy as np
from sklearn import model_selection
# from keras.preprocessing.image import ImageDataGenerator

In [2]:
# 中心から任意のサイズをトリミング
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

In [3]:
# パラメータの初期化
classes = ['beefdon', 'curry', 'fish', 'humberger', 'katsudon', 'ramen', 'seafooddon', 'soba', 'spaghetti', 'tempuradon'
         , 'udon', 'lasagna', 'hamburger_steak', 'nikujaga', 'fried_rice', 'omelette', 'sukiyaki', 'sushi', 'gyoza', 'fried_chicken'
         , 'okonomiyaki', 'croquette', 'pancake', 'french_toast', 'tart', 'taco_rice', 'doria', 'takoyaki', 'tapioca', 'yakisoba'
         , 'ginger_pork', 'steak', 'roast_meat', 'bread', 'yakitori', 'rice_ball', 'goya_chanpuru', 'instant_noodle', 'meat_bun', 'gratin'
         , 'mabo_tofu', 'tiramisu', 'cream_puff', 'eclair', 'hot_dog', 'potato_salad', 'stew', 'pizza', 'french_fry', 'avocado_salad']
num_classes = len(classes)
image_size = 150

In [4]:
# 画像の読み込みとnumpy配列への変換
X = [] # リスト
Y = [] # リスト

In [5]:
for index, classlabel in enumerate(classes):
    photos_dir = './food/' + classlabel  
    files = glob.glob(photos_dir + '/*.jpg')
    for i, file in enumerate(files):
        image = Image.open(file)
        image = image.convert('RGB')
        # image = image.convert('RGB')の前にトリミングするとメモリオーバー、理由不明
        # image = crop_center(image, 300, 300) # (500, 500)でメモリオーバー
        image = image.resize((image_size, image_size))
        data = np.asarray(image, dtype=np.float32) / 255.0  # 正規化 # dtype=np.float32でサイズ縮小、MemoryError解消のため # 2020/7/7
        X.append(data)
        Y.append(index)

X = np.array(X)
Y = np.array(Y)


/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


In [6]:
print(X.nbytes)
print(Y.nbytes)

4476600000
132640


In [7]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, Y)
# X_train, X_test = model_selection.train_test_split(X)
print(X_train.nbytes)
print(X_test.nbytes)

3357450000
1119150000


In [8]:
# メモリ解放
X = []

In [9]:
# y_train, y_test = model_selection.train_test_split(Y)
# メモリ解放
y = []
print(y_train)

[10 26 42 ... 43 10 26]


In [10]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils

y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [11]:
#　モデルの定義
model = Sequential()
model.add(Conv2D(32, (3,3), activation='relu', input_shape=(image_size, image_size, 3)))
model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# opt = SGD(lr=0.01)
opt = Adam()

model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=100)

score = model.evaluate(X_test, y_test, batch_size=32)

print(score)
model.save('./food_recognition_cnn_50.h5')

Epoch 1/100
389/389 [==============================] - 36s 93ms/step - loss: 3.8921 - accuracy: 0.0401
Epoch 2/100
389/389 [==============================] - 36s 92ms/step - loss: 3.8118 - accuracy: 0.0573
Epoch 3/100
389/389 [==============================] - 36s 92ms/step - loss: 3.5624 - accuracy: 0.1033
Epoch 4/100
389/389 [==============================] - 36s 92ms/step - loss: 3.1844 - accuracy: 0.1773
Epoch 5/100
389/389 [==============================] - 36s 92ms/step - loss: 2.7327 - accuracy: 0.2774
Epoch 6/100
389/389 [==============================] - 36s 92ms/step - loss: 2.2175 - accuracy: 0.4067
Epoch 7/100
389/389 [==============================] - 36s 92ms/step - loss: 1.7297 - accuracy: 0.5113
Epoch 8/100
389/389 [==============================] - 36s 93ms/step - loss: 1.3498 - accuracy: 0.6159
Epoch 9/100
389/389 [==============================] - 36s 92ms/step - loss: 1.0542 - accuracy: 0.6930
Epoch 10/100
389/389 [==============================] - 36s 92ms/step - l